In [11]:
!pip install mediapipe opencv-python

In [30]:
import mediapipe as mp
import cv2
import urllib.request
import os
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import time
import numpy as np
import time
import numpy as np




In [15]:
!pip install mediapipe

In [19]:
model_url = "https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task"
model_path = "pose_landmarker.task"

if not os.path.exists(model_path):
    print("Downloading model...")
    urllib.request.urlretrieve(model_url, model_path)
    print("Download complete.")
else:
    print("Model file already exists.")

Download complete.


In [23]:
model_path = '/absolute/path/to/pose_landmarker.task'

In [32]:
import cv2
import mediapipe as mp
import time
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

#Setup
model_path = 'pose_landmarker.task'
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Define the connections manually 
# Each tuple is (start_index, end_index) for bones
POSE_CONNECTIONS = [
    (11, 12), (11, 13), (13, 15), (12, 14), (14, 16), # Shoulders to wrists
    (11, 23), (12, 24), (23, 24),                   # Torso
    (23, 25), (24, 26), (25, 27), (26, 28)          # Hips to ankles
]
# Configuration
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO
)

cap = cv2.VideoCapture(0)

with PoseLandmarker.create_from_options(options) as landmarker:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        # Prepare Image
        h, w, _ = frame.shape
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
        
        # Detection
        timestamp = int(time.time() * 1000)
        result = landmarker.detect_for_video(mp_image, timestamp)

        #Custom Drawing (No landmark_pb2 required)
        if result.pose_landmarks:
            for pose in result.pose_landmarks:
                # Draw Connections (Lines)
                for connection in POSE_CONNECTIONS:
                    start = pose[connection[0]]
                    end = pose[connection[1]]
                    c1 = (int(start.x * w), int(start.y * h))
                    c2 = (int(end.x * w), int(end.y * h))
                    cv2.line(frame, c1, c2, (0, 255, 0), 2)

                # Draw Keypoints (Circles)
                for landmark in pose:
                    px = int(landmark.x * w)
                    py = int(landmark.y * h)
                    cv2.circle(frame, (px, py), 5, (0, 0, 255), -1)

        cv2.imshow('MediaPipe Pose - 2026 Tasks API', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
